# PyCity Schools Analysis
* First observed trend
* Second observed trend
* Third observed trend

In [1]:
from os import path
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
csv_file01 = path.join('..', 'raw_data', 'schools_complete.csv')
csv_file02 = path.join('..', 'raw_data', 'students_complete.csv')

In [3]:
schools_df = pd.read_csv(csv_file01)
students_df = pd.read_csv(csv_file02)

### Prepping DFs

In [4]:
# add new boolean columns to students_df that show student passing ability
pass_read_score = [1 if i >= 70 else 0 for i in students_df['reading_score']]
pass_math_score = [1 if i >= 70 else 0 for i in students_df['math_score']]

total_student_score = students_df['reading_score'] + students_df['math_score']
pass_overall_score = [1 if i >= 140 else 0 for i in total_student_score]

students_df['read_pass'] = pass_read_score
students_df['math_pass'] = pass_math_score
students_df['overall_pass'] = pass_overall_score

# rename schools_df column label
schools_df = schools_df.rename(columns={'name':'school'})


## District Summary

In [5]:
total_schools = len(schools_df['school'].unique())
total_students = schools_df['size'].sum()
total_budget = schools_df['budget'].sum()
avg_math_score = students_df['math_score'].mean()
avg_read_score = students_df['reading_score'].mean()

perc_pass_math = students_df['math_pass'].sum() / total_students
perc_pass_read = students_df['read_pass'].sum() / total_students
perc_pass_overall = students_df['overall_pass'].sum() / total_students

# format values
total_students_frmt = "{:,}".format(total_students)
total_budget_frmt = "${:,}".format(total_budget)
avg_math_score_frmt = "{:.4f}".format(avg_math_score)
avg_read_score_frmt = "{:.4f}".format(avg_read_score)
perc_list = [perc_pass_math, perc_pass_read, perc_pass_overall]
perc_formatted = ["{:.2f}%".format(perc*100) for perc in perc_list]

district_summary_df = pd.DataFrame({
    'Total Schools': total_schools,
    'Total Students': total_students_frmt,
    'Total Budget': total_budget_frmt,
    'Average Math Score': avg_math_score_frmt,
    'Average Reading Score': avg_read_score_frmt,
    '% Passing Math': perc_formatted[0],
    '% Passing Reading': perc_formatted[1],
    '% Overall Passing Rate': perc_formatted[2]
}, index=['summary values'], columns=[
    'Total Schools',
    'Total Students',
    'Total Budget',
    'Average Math Score',
    'Average Reading Score',
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing Rate'
])

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
summary values,15,"39,170","$24,649,428",78.9854,81.8778,74.98%,85.81%,89.39%


## School Summary

In [6]:
students_gb_sm_df = students_df.groupby(by='school').sum().reset_index()

school_summary_df = pd.merge(schools_df,students_gb_sm_df, how='outer', on='school')
school_summary_df = school_summary_df.rename(columns={
    'school': 'School',
    'type': 'School Type',
    'size': 'Total Students',
    'budget': 'Total School Budget',
    'reading_score': 'Average Reading Score',
    'math_score': 'Average Math Score',
    'read_pass': '% Passing Reading',
    'math_pass': '% Passing Math',
    'overall_pass': '% Overall Passing Rate'
})

school_summary_df['Per Student Budget'] = school_summary_df['Total School Budget'] / school_summary_df['Total Students']
school_summary_df['Average Math Score'] = school_summary_df['Average Math Score'] / school_summary_df['Total Students']
school_summary_df['Average Reading Score'] = school_summary_df['Average Reading Score'] / school_summary_df['Total Students']
school_summary_df['% Passing Reading'] = school_summary_df['% Passing Reading'] / school_summary_df['Total Students']
school_summary_df['% Passing Math'] = school_summary_df['% Passing Math'] / school_summary_df['Total Students']
school_summary_df['% Overall Passing Rate'] = school_summary_df['% Overall Passing Rate'] / school_summary_df['Total Students']

# transfer of ownership
clean_ss_df = school_summary_df.copy()

# format columns
clean_ss_df['Total Students'] = clean_ss_df['Total Students'].map("{:,}".format)
clean_ss_df['Total School Budget'] = clean_ss_df['Total School Budget'].map("${:,}".format)
clean_ss_df['Per Student Budget'] = clean_ss_df['Per Student Budget'].map("${:,.2f}".format)
clean_ss_df['Average Math Score'] = clean_ss_df['Average Math Score'].map("{:.4f}".format)
clean_ss_df['Average Reading Score'] = clean_ss_df['Average Reading Score'].map("{:.4f}".format)
clean_ss_df['% Passing Math'] = (clean_ss_df['% Passing Math']*100).map("{:.2f}%".format)
clean_ss_df['% Passing Reading'] = (clean_ss_df['% Passing Reading']*100).map("{:.2f}%".format)
clean_ss_df['% Overall Passing Rate'] = (clean_ss_df['% Overall Passing Rate']*100).map("{:.2f}%".format)


# filter and order columns
clean_ss_df = clean_ss_df[[
    'School','School Type','Total Students','Total School Budget',
    'Per Student Budget','Average Math Score','Average Reading Score',
    '% Passing Math','% Passing Reading','% Overall Passing Rate'
]].set_index('School')
del clean_ss_df.index.name
clean_ss_df


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Huang High School,District,"2,917","$1,910,635",$655.00,76.6294,81.1827,65.68%,81.32%,84.98%
Figueroa High School,District,"2,949","$1,884,411",$639.00,76.7118,81.1580,65.99%,80.74%,84.67%
Shelton High School,Charter,"1,761","$1,056,600",$600.00,83.3595,83.7257,93.87%,95.85%,99.38%
Hernandez High School,District,"4,635","$3,022,020",$652.00,77.2898,80.9344,66.75%,80.86%,84.88%
Griffin High School,Charter,"1,468","$917,500",$625.00,83.3515,83.8168,93.39%,97.14%,99.46%
Wilson High School,Charter,"2,283","$1,319,574",$578.00,83.2742,83.9895,93.87%,96.54%,99.26%
Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.0619,83.9758,94.13%,97.04%,99.57%
Bailey High School,District,"4,976","$3,124,928",$628.00,77.0484,81.0340,66.68%,81.93%,85.19%
Holden High School,Charter,427,"$248,087",$581.00,83.8033,83.8150,92.51%,96.25%,98.59%
Pena High School,Charter,962,"$585,858",$609.00,83.8399,84.0447,94.59%,95.95%,99.17%


## Top Performing Schools (By Passing Rate)

In [7]:
clean_ss_df.sort_values(by='% Overall Passing Rate', ascending=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.0619,83.9758,94.13%,97.04%,99.57%
Griffin High School,Charter,"1,468","$917,500",$625.00,83.3515,83.8168,93.39%,97.14%,99.46%
Shelton High School,Charter,"1,761","$1,056,600",$600.00,83.3595,83.7257,93.87%,95.85%,99.38%
Wilson High School,Charter,"2,283","$1,319,574",$578.00,83.2742,83.9895,93.87%,96.54%,99.26%
Wright High School,Charter,"1,800","$1,049,400",$583.00,83.6822,83.9550,93.33%,96.61%,99.22%


## Bottom Performing Schools (By Passing Rate)

In [8]:
clean_ss_df.sort_values(by='% Overall Passing Rate', ascending=True).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Figueroa High School,District,"2,949","$1,884,411",$639.00,76.7118,81.1580,65.99%,80.74%,84.67%
Rodriguez High School,District,"3,999","$2,547,363",$637.00,76.8427,80.7447,66.37%,80.22%,84.75%
Ford High School,District,"2,739","$1,763,916",$644.00,77.1026,80.7463,68.31%,79.30%,84.78%
Hernandez High School,District,"4,635","$3,022,020",$652.00,77.2898,80.9344,66.75%,80.86%,84.88%
Huang High School,District,"2,917","$1,910,635",$655.00,76.6294,81.1827,65.68%,81.32%,84.98%


## Math Scores by Grade

In [9]:
school_math_scores_df = students_df.groupby(by=['school','grade'])[['math_score']].mean()
school_math_scores_df = school_math_scores_df.pivot_table(index='school', columns='grade', values='math_score')
school_math_scores_df = school_math_scores_df.rename_axis(None).rename_axis(None, axis=1)

# format values
grade_level = ['9th','10th','11th','12th']
for i in range(len(grade_level)):
    school_math_scores_df[grade_level[i]] = school_math_scores_df[grade_level[i]].map("{:.4f}".format)

school_math_scores_df[['9th', '10th', '11th', '12th']]

,9th,10th,11th,12th
Bailey High School,77.0837,76.9968,77.5156,76.4922
Cabrera High School,83.0947,83.1545,82.7656,83.2775
Figueroa High School,76.4030,76.5400,76.8843,77.1514
Ford High School,77.3613,77.6723,76.9181,76.1800
Griffin High School,82.0440,84.2291,83.8421,83.3562
Hernandez High School,77.4385,77.3374,77.1360,77.1866
Holden High School,83.7874,83.4298,85.0000,82.8554
Huang High School,77.0273,75.9087,76.4466,77.2256
Johnson High School,77.1879,76.6911,77.4917,76.8632
Pena High School,83.6255,83.3720,84.3281,84.1215


## Reading Scores by Grade

In [10]:
school_read_scores_df = students_df.groupby(by=['school','grade'])[['reading_score']].mean()
school_read_scores_df = school_read_scores_df.pivot_table(index='school', columns='grade', values='reading_score')
school_read_scores_df = school_read_scores_df.rename_axis(None).rename_axis(None, axis=1)

# format values
grade_level = ['9th','10th','11th','12th']
for i in range(len(grade_level)):
    school_read_scores_df[grade_level[i]] = school_read_scores_df[grade_level[i]].map("{:.4f}".format)

school_read_scores_df[['9th', '10th', '11th', '12th']]

,9th,10th,11th,12th
Bailey High School,81.3032,80.9072,80.9456,80.9125
Cabrera High School,83.6761,84.2532,83.7884,84.2880
Figueroa High School,81.1986,81.4089,80.6403,81.3849
Ford High School,80.6327,81.2627,80.4036,80.6623
Griffin High School,83.3692,83.7069,84.2881,84.0137
Hernandez High School,80.8669,80.6601,81.3961,80.8571
Holden High School,83.6772,83.3246,83.8155,84.6988
Huang High School,81.2903,81.5124,81.4175,80.3060
Johnson High School,81.2607,80.7734,80.6160,81.2276
Pena High School,83.8073,83.6120,84.3359,84.5912


## Scores by School Spending

### dynamic bin and label generator for per student spending ranges

In [11]:
# locate the per student spending max and mins
spend_min = school_summary_df['Per Student Budget'].min()
spend_max = school_summary_df['Per Student Budget'].max()

# dynamic bin generator
bins = list(np.linspace(spend_min - 1,spend_max + 1,num=(spend_max-spend_min)/14, dtype=int))

# dynamic label generator
labels = ["<${}".format(bins[1])]
for k in range(len(bins)):
    labels.append("${0}-{1}".format(bins[k-1], bins[k]))
del labels[1:3]

In [12]:
school_summary_df['Spending Ranges (Per Student)'] = pd.cut(school_summary_df['Per Student Budget'], bins=bins, labels=labels)
school_spending_ranges_df = school_summary_df[[
    'Spending Ranges (Per Student)','Average Math Score','Average Reading Score',
    '% Passing Math','% Passing Reading','% Overall Passing Rate']].copy()

# grouby Spending Ranges (Per Student)
school_spend_gp_mn_df = school_spending_ranges_df.groupby(by='Spending Ranges (Per Student)').mean()

# format columns
school_spend_gp_mn_df['% Passing Math'] = (school_spend_gp_mn_df['% Passing Math']*100).map("{:.2f}%".format)
school_spend_gp_mn_df['% Passing Reading'] = (school_spend_gp_mn_df['% Passing Reading']*100).map("{:.2f}%".format)
school_spend_gp_mn_df['% Overall Passing Rate'] = (school_spend_gp_mn_df['% Overall Passing Rate']*100).map("{:.2f}%".format)

school_spend_gp_mn_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$596,83.455399,83.933814,93.46%,96.61%,99.16%
$596-616,83.599686,83.885211,94.23%,95.90%,99.27%
$616-636,80.199966,82.425360,80.04%,89.54%,92.32%
$636-656,77.866721,81.368774,70.35%,83.00%,86.87%


## Scores by School Size

### dynamic bin and label generator for school size

In [13]:
# locate the per student schooling max and mins
school_min = school_summary_df['Total Students'].min()
school_max = school_summary_df['Total Students'].max()

# dynamic bin generator
bins = list(np.linspace(school_min - 1,school_max + 1,num=(school_max-school_min)/1000, dtype=int))

# dynamic label generator
labels = ["(<{})".format(bins[1])]
for k in range(len(bins)):
    labels.append("({0}-{1})".format(bins[k-1], bins[k]))
del labels[1:3]

labels[0] = "Small {}".format(labels[0])
labels[1] = "Medium {}".format(labels[1])
labels[2] = "Large {}".format(labels[2])

In [14]:
school_summary_df['School Size'] = pd.cut(school_summary_df['Total Students'], bins=bins, labels=labels)
school_size_ranges_df = school_summary_df[[
    'School Size','Average Math Score','Average Reading Score',
    '% Passing Math','% Passing Reading','% Overall Passing Rate']].copy()

# grouby School Size (Per Student)
school_size_gp_mn_df = school_size_ranges_df.groupby('School Size').mean()

# format columns
school_size_gp_mn_df['% Passing Math'] = (school_size_gp_mn_df['% Passing Math']*100).map("{:.2f}%".format)
school_size_gp_mn_df['% Passing Reading'] = (school_size_gp_mn_df['% Passing Reading']*100).map("{:.2f}%".format)
school_size_gp_mn_df['% Overall Passing Rate'] = (school_size_gp_mn_df['% Overall Passing Rate']*100).map("{:.2f}%".format)

school_size_gp_mn_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1943),83.502373,83.883125,93.59%,96.59%,99.21%
Medium (1943-3460),78.429493,81.769122,73.46%,84.47%,88.42%
Large (3460-4977),77.063340,80.919864,66.46%,81.06%,84.95%


## Scores by School Type

In [15]:
school_type_gp_mn_df = school_summary_df.groupby(by='School Type').mean()

school_type_df = school_type_gp_mn_df[[
    'Average Math Score','Average Reading Score','% Passing Math',
    '% Passing Reading','% Overall Passing Rate']].copy()

# format columns
school_type_df['% Passing Math'] = (school_type_df['% Passing Math']*100).map("{:.2f}%".format)
school_type_df['% Passing Reading'] = (school_type_df['% Passing Reading']*100).map("{:.2f}%".format)
school_type_df['% Overall Passing Rate'] = (school_type_df['% Overall Passing Rate']*100).map("{:.2f}%".format)

school_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.62%,96.59%,99.22%
District,76.956733,80.966636,66.55%,80.80%,84.89%
